<font face="微软雅黑" color=green size=5>Julia程序效率</font>

# Julia的语言特点

Julia语言是一种历史很短的计算机语言， 公开发布于2012年。 其设计理念就是希望兼有Python、R、Matlab这样的动态语言的易用性， 以及C、C++、Java这样的静态语言的运行速度。 所以Julia很适合用来做统计和金融计算。

Julia语言的特点有：

动态语言；  
使用基于LLVM的动态编译技术，可以动态生成高效的运行代码；  
不需要声明类型也一般能够产生高效代码， 需要时声明类型可以提高效率。  

# 程序性能评测

## 程序计时

用@time宏对一个表达式计时和查看内存分配情况，如:

In [1]:
@time sqrt.(rand(10_000_000));

  0.113557 seconds (4 allocations: 152.588 MiB, 14.56% gc time)


结果中有程序运行时间的秒数， 进行了10次内存分配， 总计分配的内存数（中间可能也有释放的内存）。

@time宏执行一个表达式， 显示运行时间、分配内存、垃圾收集时间比例， 然后返回表达式的结果。

@time也可以对一个结构计时，如

In [2]:
s = 0.0
@time for i=1:10_000_000
    global s
    s += sqrt(rand())
end

  1.398011 seconds (20.00 M allocations: 305.176 MiB, 12.31% gc time)


这个程序慢了很多，分配内存的次数和总量也比较多。

还有一个@timev宏与@time类似， 提供的信息更多。 @time和@timev仅是显示表达式执行的时间和内存分配信息， 不影响表达式返回结果， 所以在程序中使用这两个函数计时一般不影响程序的结果。

## 程序瓶颈查找(profiler)

为了提高程序效率， 并不是所有的程序代码都需要仔细地考察并改进效率。 大部分的程序可能仅仅执行一两次， 可能仅需要执行几秒钟到几分钟。 这样的程序不需要想办法提高执行效率， 只要程序结果正确就可以了。

有些程序需要执行许多次， 可能会提供给许多用户使用， 这样的程序即使耗时并不多， 也有必要去设法改进其运行效率。 有些程序虽然不需要执行许多次， 但是运行所需的时间可能很长， 比如几小时到几天， 这样的程序需要设法改进效率。

**对一段比较长的程序， 如果需要改进效率， 并不需要考虑所有的代码行， 而是只要改进程序执行消耗时间最多的部分就可以了， 这样的部分称为程序的瓶颈**。 Julia内置了一个profiler工具， 用定时抽查的方式查看那些代码行被执行得最多。 这样的工具在使用时不需要修改源程序， 对运行速度的影响也很小。

因为Julia使用了即时编译， 所以一个函数在第一次被调用时需要有额外的编译时间， 第二次调用就没有这个问题了。 在用profiler分析一个函数是应该从第二次调用开始分析。

在执行程序之前，调用:

using Profile  
Profile.clear()  
然后执行要分析的程序。 执行完后调用：  

Profile.print()  
将显示程序的执行次数， 包括调用的库函数的执行次数， 执行次数多的程序语句是需要优化的。

## BenchmarkTools

为了对运行时间较短的程序计时， 通常会人为地重复该程序。 BenchmarkTools提供了更完善的计时统计， 以及系统的性能比较(benchmark)能力。

例如，如下的简短程序：

In [5]:
A = rand(2)
B = rand(2)
C = zeros(2)
C = A + B

2-element Vector{Float64}:
 0.698714491968055
 1.075390843171893

对第二行计时如下：

In [4]:
using BenchmarkTools
@benchmark C = A + B

BenchmarkTools.Trial: 10000 samples with 888 evaluations.
 Range (min … max):  63.401 ns …  3.095 μs  ┊ GC (min … max): 0.00% … 95.20%
 Time  (median):     71.284 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   88.948 ns ± 86.021 ns  ┊ GC (mean ± σ):  3.47% ±  3.70%

  █▅▂▃▄▇  ▂ ▅▁▁    ▆▁▂▃ ▄▄ ▂▂      ▂                   ▄▆▁    ▂
  █████████████████████▇██▇██████▇████▇▆▆▆▆▇▇▆▇▆▅▇█▅▅▃▄███▆▇█ █
  63.4 ns      Histogram: log(frequency) by time       147 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

换用加点写法：

In [5]:
@benchmark C .= A .+ B

BenchmarkTools.Trial: 10000 samples with 199 evaluations.
 Range (min … max):  428.141 ns …   8.437 μs  ┊ GC (min … max): 0.00% … 93.28%
 Time  (median):     471.859 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   574.246 ns ± 215.402 ns  ┊ GC (mean ± σ):  0.26% ±  1.32%

  █  ▃                                                           
  █▅▅█▅▃▃▂▃▂▅▃▃▂▂▂▂▆▃▃▃▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▃▃▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▄▅▃▃▂▂ ▃
  428 ns           Histogram: frequency by time          997 ns <

 Memory estimate: 64 bytes, allocs estimate: 2.

这个版本性能更差。

换用显式循环：

In [6]:
@benchmark for i=1:2
    C[i] = A[i] + B[i]
end

BenchmarkTools.Trial: 10000 samples with 308 evaluations.
 Range (min … max):  195.779 ns …   4.334 μs  ┊ GC (min … max): 0.00% … 92.43%
 Time  (median):     216.883 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   283.948 ns ± 147.607 ns  ┊ GC (mean ± σ):  0.66% ±  2.27%

  █▅▃▁▃▃▂▁▅▃▂▁▅▂▂▂▁ ▁   ▂▄▁▁ ▁▂▁   ▂▃▂▂▁                      ▁ ▂
  █████████████████▇█▆█▅█████████▇▅█████▇▆▆▇▆▅▅▆▄▄▄▁▃▅▄▅▅▄▅▄▄▅█ █
  196 ns        Histogram: log(frequency) by time        757 ns <

 Memory estimate: 96 bytes, allocs estimate: 6.

不如向量化版本， 但优于加点版本。

以上的程序是在全局名字空间（命令行对应的名字空间）中执行的。 如果调用函数， 结果则不同：

In [7]:
A = rand(2); B = rand(2); C = zeros(2);

function vecadd1(a,b,c)
    c = a + b
    return nothing
end

function vecadd2(a,b,c)
    c .= a .+ b
    return nothing
end

function vecadd3(a,b,c)
    for i=1:length(c)
        c[i] = a[i] + b[i]
    end
    return nothing
end
@benchmark vecadd1(A, B, C)

BenchmarkTools.Trial: 10000 samples with 921 evaluations.
 Range (min … max):  63.301 ns …   3.598 μs  ┊ GC (min … max): 0.00% … 95.61%
 Time  (median):     78.610 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   95.601 ns ± 105.370 ns  ┊ GC (mean ± σ):  4.05% ±  3.84%

  █▄▄▄▁  ▅▁▃▁ ▄▁▂ ▁▁▅▅▁▁▁▁▂  ▅▂▁▃        ▆▃ ▃▂                 ▂
  ██████▇████████▇████████████████▇▆▆▇▆▆██████▆▆▆▄▄▅▄▅▅▅▁▅▅▄▄▅ █
  63.3 ns       Histogram: log(frequency) by time       179 ns <

 Memory estimate: 80 bytes, allocs estimate: 1.

In [8]:
@benchmark vecadd2(A, B, C)

BenchmarkTools.Trial: 10000 samples with 990 evaluations.
 Range (min … max):  32.727 ns … 159.798 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     33.333 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   35.243 ns ±   6.788 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▁▄▃▂▁▁ ▂ ▁▁      ▂                                         ▁
  ████████▆█▆██▇▅▅█▆██▇▆▅▇▆▇▆▇▇▆▆▆▅▆▆▆▆▃▆▅▅▄▁▃▁▁▁▃▃▄▄▅▆▃▅▆▆▄▆█ █
  32.7 ns       Histogram: log(frequency) by time      72.9 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [9]:
@benchmark vecadd3(A, B, C)

BenchmarkTools.Trial: 10000 samples with 994 evaluations.
 Range (min … max):  25.956 ns … 316.901 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     28.169 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   35.774 ns ±  13.267 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▂▃▃▂▂▄▁ ▂▁▃    ▂       ▆▁    ▄▃    ▁                        ▁
  ████████████▆▄▅▅█▇▄▃▅▃▃▄██▅▆▆▆██▇▄▆▆█▆▄▅▃▃▃▄▄▂▄▄▄▂▂▃▄▃▃▃▃▃▄▅ █
  26 ns         Histogram: log(frequency) by time      80.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

加点和显式循环的版本性能相近， 都优于向量化版本， 比命令行版本也提高了很多。 这可能与即时编译系统对函数的优化有关。

# 数据类型与运行效率

## 类型声明

Julia允许对函数自变量、函数返回值、函数的局部变量声明数据类型， 但是一般不需要声明也能产生运行效率很好的执行代码。 声明方式是在变量名后面用双冒号::后面跟类型名，如

In [6]:
function f_tp01(x::Float64)::Float64
    local y::Float64
    y = (1 + x)^2
    y = sin(y)
    return y
end
f_tp01(1.5)

-0.03317921654755682

说明类型有可能会提高程序效率， 但是也限制了函数调用方法， 尤其是整型与实型会看成两种不同类型，所以：

f_tp01(1)  
结果：  

MethodError: no method matching f_tp01(::Int64)  
Closest candidates are:  
  f_tp01(!Matched::Float64) at In[12]:2  
 
Stacktrace:  
 [1] top-level scope at In[13]:1  
上面的调用错误是因为f_tp01()函数只允许浮点型自变量， 而1不是浮点型, Julia在调用函数时不会自动将整型转换成浮点型。 因为Julia的四则运算的设计是允许将整型与浮点型混合运算的， 所以四则运算不存在这样的问题。

函数定义时如果不声明自变量类型， 则不存在这个问题，如：

In [7]:
function f_tp01b(x)
    y = (1 + x)^2
    y = sin(y)
    return y
end
f_tp01b(1.5)

-0.03317921654755682

In [12]:
f_tp01b(1)

-0.7568024953079282

所以自定义函数中如非必要并不是所有自变量、返回值、局部变量都声明类型就更好。

为了使得f01()能够对其它数也适用， 可以写一个更一般的类型作为自变量然后转换成浮点型，如：

In [8]:
function f_tp01(x::Number)::Float64
    local y::Float64
    y = (1 + Float64(x))^2
    y = sin(y)
    return y
end
f_tp01(1)

-0.7568024953079282

在定义变量时，要尽量保持与其后面运算时的类型一致，有几个函数可以帮助完成这种定义。  
zero(value)    
eltype(array)  
one(value)  
similar(array)  
在定义变量时，可以使用这4个函数来避免常犯的错误。

In [15]:
pos(x) = x < 0 ? zero(x) : x #zero(x)使与x类型保持一致

pos (generic function with 1 method)

In [19]:
typeof(pos(-2.3))

Float64

## 多重派发

Julia的同一个函数可以根据自变量个数和自变量类型的不同而执行不同的操作， 称为“多重派发”（multiple dispatch）。 这有些像是面向对象语言的方法重载（overloading）， 但是：

    ·方法重载是编译时根据对象的类决定的；  
    ·方法重载仅能根据其依附的对象而选择对应的方法，即所谓单重派发；  
    ·Julia是动态语言，其函数调用是运行时动态决定的；  
    ·Julia可以根据其多个自变量的类型而选取不同方法。  
多重派发可以让多个类似的操作共享同一个函数名。 比如， 可以这样设计绘图函数plot()， 可以仅输入一个数值型向量y， 作序列图； 可以输入两个数值型向量x和y，作散点图； 可以输入一个字符串向量x和数值型向量y， 作条形图。 不同的自变量个数和类型决定了要执行的操作， 这些操作称为plot()函数的“方法”。

## 代码检查

### code_warntype

Julia不要求声明函数自变量、返回值和局部变量类型， 但是如果函数返回值不能被其输入数据类型确定， 这样的函数很难得到高效的执行代码。 这种要求称为程序的“类型稳定性”。

例如，下面的程序：

In [14]:
function f_tp02(x)
    if x >= 0
        return sqrt(x)
    else
        return "Negative number have no square root!"
    end
end

f_tp02 (generic function with 1 method)

这个函数从表面上看很正常， 但是违反了类型稳定性。 因为不论输入x为什么数， 返回值的类型都依赖于x的数值而不能由x的数据类型决定， 这就使得依赖于返回值的代码无法确定返回值类型从而无法优化。

用宏 **@code_warntype**可以检查一个函数调用中的类型稳定性问题，如

In [15]:
@code_warntype f_tp02(1.5)

MethodInstance for f_tp02(::Float64)
  from f_tp02(x) in Main at In[14]:1
Arguments
  #self#::Core.Const(f_tp02)
  x::Float64
Body::Union{Float64, String}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─ %3 = Main.sqrt(x)::Float64
└──      return %3
3 ─      return "Negative number have no square root!"



结果中的类型不稳定问题会用红色显示。

另外，如果一个函数反复被调用或者内部有很多次循环， 某些局部变量在多次调用或者多次循环中不能预先确定类型或者类型被改变， 也会造成程序性能急剧下降。 如：

In [16]:
function f_tp03(x)
  s = 0
  for xi in x
    s += sqrt(xi)
  end 
  return s
end

f_tp03 (generic function with 1 method)

变量s开始是整型，但在循环中变成了浮点型。初始化可写成s = 0.0。

### code_lowerd

**用@code_lowerd可以查看程序底层运行过程。**

In [5]:
cal(a, b, c) = a + b * c

cal (generic function with 1 method)

In [6]:
@code_lowered cal(1, 2, 3)

CodeInfo(
1 ─ %1 = b * c
│   %2 = a + %1
└──      return %2
)

### code_typed

**用@code_typed可以程序查看运行时类型的变化。**

In [10]:
cal(a, b, c) = a + b * c

cal (generic function with 1 method)

In [7]:
@code_typed cal(1, 2, 3.0)

CodeInfo(
1 ─ %1 = Base.sitofp(Float64, b)::Float64
│   %2 = Base.mul_float(%1, c)::Float64
│   %3 = Base.sitofp(Float64, a)::Float64
│   %4 = Base.add_float(%3, %2)::Float64
└──      return %4
) => Float64

In [8]:
@code_typed cal(1, 2, 3)

CodeInfo(
1 ─ %1 = Base.mul_int(b, c)::Int64
│   %2 = Base.add_int(a, %1)::Int64
└──      return %2
) => Int64

### code_llvm

**用@code_llvm查看llvm编译器的运行过程。**

In [11]:
cal(a, b, c) = a + b * c

cal (generic function with 1 method)

In [12]:
@code_llvm cal(1, 2, 3)

;  @ In[11]:1 within `cal`
; Function Attrs: uwtable
define i64 @julia_cal_2026(i64 signext %0, i64 signext %1, i64 signext %2) #0 {
top:
; ┌ @ int.jl:88 within `*`
   %3 = mul i64 %2, %1
; └
; ┌ @ int.jl:87 within `+`
   %4 = add i64 %3, %0
; └
  ret i64 %4
}


### code_native

**用@code_native查看机器语言的运行过程。**

In [13]:
cal(a, b, c) = a + b * c

cal (generic function with 1 method)

In [14]:
@code_native cal(1, 2, 3)

	.text
; ┌ @ In[13]:1 within `cal`
	pushq	%rbp
	movq	%rsp, %rbp
; │┌ @ int.jl:88 within `*`
	imulq	%r8, %rdx
; │└
; │┌ @ int.jl:87 within `+`
	leaq	(%rdx,%rcx), %rax
; │└
	popq	%rbp
	retq
	nop
; └


# 函数的效率

由于Julia的多重派发、动态编译特性， Julia程序最好分解成短小、输入类型明确的多个函数。 函数调用的成本很低， 动态编译可以将函数编译成针对特定输入的高效代码。

## 全局变量的问题

为了运行效率考虑， 应尽量避免使用全局变量。 全局变量的取值乃至于类型都可能被不同位置的代码改变， 使得编译器很难预估全局变量的类型， 造成编译代码低效。 尽可能使用局部变量与函数自变量保存和传递数据。 Julia的函数是按引用进行参数传递的， 所以数组的传递不会制作副本， 没有内存和时间的额外开销。

**Julia的编译器优化主要是针对函数， 所以对效率敏感的代码都应该写成函数而不是直接在全局变量空间执行。**

用const声明的全局变量不能改变类型， 这样的全局变量对程序效率的影响较小。

考虑下面的利用了全局变量作为选项的程序：

In [17]:
POWER=2
function f_gl01(x::Vector{Float64})
    s = 0.0
    for xi in x
        s += xi ^ POWER
    end
    return s
end
f_gl01(rand(10))

using BenchmarkTools
@benchmark f_gl01(rand(100_000))

BenchmarkTools.Trial: 537 samples with 1 evaluation.
 Range (min … max):  5.525 ms … 18.846 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.278 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   9.305 ms ±  3.030 ms  ┊ GC (mean ± σ):  2.26% ± 5.33%

   ██ ▇▄█▇▅▄▁▅  ▂                    ▁▂                       
  ▄██▇████████▆██▆▇▇▇▆▄▇▅▄▃▆▄▃▄▃▄▃▅▆▅███▇█▅▄▄▃▃▃▃▄▄▃▂▁▃▃▁▂▂▂ ▄
  5.52 ms        Histogram: frequency by time        17.2 ms <

 Memory estimate: 5.34 MiB, allocs estimate: 300002.

将上面的POWER声明为常数可以提高效率。 Julia用const关键字声明全局变量常数， 但是这种全局变量仅仅是不允许改变数据类型， 变量值还是允许改变的， 改变变量值会有警告信息。

修改后的程序如：

In [18]:
const P_const=2
function f_gl01b(x::Vector{Float64})
    s = 0.0
    for xi in x
        s += xi ^ P_const
    end
    return s
end
f_gl01b(rand(10))

using BenchmarkTools
@benchmark f_gl01b(rand(100_000))

BenchmarkTools.Trial: 7076 samples with 1 evaluation.
 Range (min … max):  226.600 μs …   8.722 ms  ┊ GC (min … max):  0.00% … 86.73%
 Time  (median):     609.600 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   702.124 μs ± 603.147 μs  ┊ GC (mean ± σ):  10.53% ± 11.25%

      █▅▂                                                        
  ▄▂▂▇███▆▄▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  227 μs           Histogram: frequency by time         4.59 ms <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

比原来效率提高了一个数量级。

在调用全局变量的位置标识其类型也可以提高效率，如

In [19]:
POWER=2
function f_gl01c(x::Vector{Float64})
    s = 0.0
    for xi in x
        s += xi ^ POWER::Int
    end
    return s
end
f_gl01c(rand(10))

using BenchmarkTools
@benchmark f_gl01c(rand(100_000))

BenchmarkTools.Trial: 8952 samples with 1 evaluation.
 Range (min … max):  220.700 μs …   6.826 ms  ┊ GC (min … max): 0.00% … 84.62%
 Time  (median):     501.900 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   554.771 μs ± 452.155 μs  ┊ GC (mean ± σ):  8.79% ± 10.66%

  █    ▄█▁                                                       
  █▆▄▂▃███▇▅▄▃▃▃▃▂▂▃▄▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂▂ ▃
  221 μs           Histogram: frequency by time         2.95 ms <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

当然，这个问题实际上完全不需要使用全局变量， 只要作为函数自变量即可：

In [20]:
function f_gl01d(x::Vector{Float64}, p)
    s = 0.0
    for xi in x
        s += xi ^ p
    end
    return s
end
f_gl01d(rand(10), 2)

using BenchmarkTools
@benchmark f_gl01d(rand(100_000), 2)

BenchmarkTools.Trial: 9113 samples with 1 evaluation.
 Range (min … max):  249.000 μs …   6.928 ms  ┊ GC (min … max): 0.00% … 83.46%
 Time  (median):     511.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   544.909 μs ± 414.139 μs  ┊ GC (mean ± σ):  7.79% ± 10.31%

  █     ▅▂                                                       
  █▅▄▄▃▂██▅▃▃▆▃▃▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂ ▃
  249 μs           Histogram: frequency by time         2.62 ms <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

用@code_warntype分析f_gl01()的问题：

In [21]:
@code_warntype f_gl01(rand(10))

MethodInstance for f_gl01(::Vector{Float64})
  from f_gl01(x::Vector{Float64}) in Main at In[17]:2
Arguments
  #self#::Core.Const(f_gl01)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  xi::Float64
Body::Any
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (xi = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (xi ^ Main.POWER)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



输出中的红色Any都提示有类型不确定问题。修改然后再用@code_warntype分析：

In [22]:
POWER=2
function f_gl01e(x::Vector{Float64})::Float64
    s::Float64 = 0.0
    for xi in x
        s += xi ^ POWER::Int
    end
    return s
end
f_gl01e(rand(10))

using BenchmarkTools
@benchmark f_gl01e(rand(100_000))

BenchmarkTools.Trial: 8873 samples with 1 evaluation.
 Range (min … max):  251.100 μs …   7.783 ms  ┊ GC (min … max): 0.00% … 83.20%
 Time  (median):     439.200 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   558.134 μs ± 418.896 μs  ┊ GC (mean ± σ):  6.75% ±  9.56%

  █▆▃▃                                                           
  ████▅█▄▇▆▆▅▅▅▅▄▃▃▃▃▄▃▂▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  251 μs           Histogram: frequency by time          2.7 ms <

 Memory estimate: 781.30 KiB, allocs estimate: 2.

In [23]:
@code_warntype f_gl01e(rand(10))

MethodInstance for f_gl01e(::Vector{Float64})
  from f_gl01e(x::Vector{Float64}) in Main at In[22]:2
Arguments
  #self#::Core.Const(f_gl01e)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  xi::Float64
Body::Float64
1 ─ %1  = Main.Float64::Core.Const(Float64)
│   %2  = Base.convert(Main.Float64, 0.0)::Core.Const(0.0)
│         (s = Core.typeassert(%2, Main.Float64))
│   %4  = x::Vector{Float64}
│         (@_3 = Base.iterate(%4))
│   %6  = (@_3 === nothing)::Bool
│   %7  = Base.not_int(%6)::Bool
└──       goto #4 if not %7
2 ┄ %9  = @_3::Tuple{Float64, Int64}
│         (xi = Core.getfield(%9, 1))
│   %11 = Core.getfield(%9, 2)::Int64
│   %12 = s::Float64
│   %13 = xi::Float64
│   %14 = Core.typeassert(Main.POWER, Main.Int)::Int64
│   %15 = (%13 ^ %14)::Float64
│   %16 = (%12 + %15)::Float64
│   %17 = Base.convert(Main.Float64, %16)::Float64
│         (s = Core.typeassert(%17, Main.Float64))
│         (@_3 = Base.iterate(%4, %11))
│   %20 = (@_3 ===

结果中没有需要改进的代码了。

## 行内函数(inline functions)

**Julia编译器自动将较小的类型明确的函数调用变成行内函数**， 即被调用的函数的实际代码被转移到调用者内部。 行内函数提高了程序效率但是会使得程序变大。

Julia自动确定哪些函数变成行内函数， 然而， 有些反复调用的代码， 尤其是用在循环和内存循环的代码， 比如数组用下标访问元素的函数， 最好人为地规定成行内函数。 **用@inline宏说明一个函数定义就可以规定其变成行内函数**。 如

## 用宏提高效率

宏(macro)是类似C语言预处理的语言构件。 宏可以生成程序代码， 可以对程序代码进行操作， 比如前面的@time、@code_warntype等。

**宏可以用来对重复性的代码进行简写， 也可以通过将运行时才执行的任务提前到代码编译阶段完成从而提高效率**。

以多项式求值的函数为例。 多项式的系数按升序保存在一维数组中。 按照通常的写法，程序为：

In [24]:
function f_ma01(x, a...)
    p = zero(x)
    for i = 1: length(a)
        p += a[i] * x ^(i-1)
    end
    return p
end

f_ma01 (generic function with 1 method)

这里没有用秦九韶法进行算法优化，直接用了幂函数。 对多项式 $f(x)=1+2x+3x^2+4x^3$ ， 可以用f_ma01()定义如下的多项式函数：

In [25]:
f_ma01b(x) = f_ma01(x, 1, 2, 3, 4)

f_ma01b (generic function with 1 method)

调用如

In [26]:
f_ma01b(1.0)

10.0

In [27]:
f_ma01b(1.5)

24.25

In [28]:
f_ma01b(1)

10

测试程序效率如下：

In [29]:
@benchmark f_ma01b(1.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.100 ns … 52.100 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.200 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.425 ns ±  0.815 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █   ▇                                                       
  █▁▁▁█▁▁▁▃▁▁▁▂▁▁▁▂▁▁▁▄▁▁▁▆▁▁▁▅▁▁▁▁▅▁▁▁▄▁▁▁▃▁▁▁▂▁▁▁▂▁▁▁▂▁▁▁▂ ▂
  2.1 ns         Histogram: frequency by time         3.5 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

将程序改成用秦九韶法， 比如 $f(x)=1+2x+3x^2+4x^3$  写成
$$f(x)=1+x(2+x(3+4x))$$
这样可以减少乘法运算的次数。 算法每一步将前一步的结果乘以x， 加上降序的后一个系数。 将f_ma01()用这种方法改写：

In [30]:
function f_ma02(x, a...)
    p = zero(x)
    for i = length(a):-1:1
        p = a[i] + p*x
    end
    return p
end

f_ma02 (generic function with 1 method)

多项式 $f(x)=1+2x+3x^2+4x^3$  的函数为：

In [31]:
f_ma02b(x) = f_ma02(x, 1, 2, 3, 4)
f_ma02b(1.5)
@benchmark f_ma02b(1.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.800 ns … 19.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.200 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.257 ns ±  0.597 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▅       █   █   ▃   ▃   ▂  ▃   ▄   ▃   ▃   ▃   ▁        ▂
  ▃▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁█▁▁▁▇▁▁▇ █
  1.8 ns       Histogram: log(frequency) by time      3.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

两个函数性能相同， 可能是Julia本身进行了优化。

这个算法还是不够优化， 因为每次调用f_ma02()时都需要访问系数数组， 而实际的多项式的系数是常数， 所以， 直接按 $f(x)=1+x(2+x(3+4x))$ 计算效率会更高，如：

In [32]:
f_ma03(x) = 1 + x*(2 + x*(3 + 4 * x))
f_ma03(1.5)
@benchmark f_ma03(1.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.001 ns … 0.300 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.001 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.043 ns ± 0.049 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                                       ▄  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  0.001 ns       Histogram: frequency by time        0.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

第三个版本效率提升了三个数量级。 问题是f_ma03()函数是完全根据多项式阶数与系数手写代码实现的， 我们需要就这个手写代码的过程用Julia程序完成， 这就是宏的作用。

一般的生成一个多项式函数的宏如下：

In [33]:
macro mac_01(x, a...)
    ex = esc(a[end])
    for i = length(a)-1:-1:1
        ex = :(muladd(t, $ex, $(esc(a[i]))))
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@mac_01 (macro with 1 method)

生成多项式函数如:

In [34]:
f_ma04(x) = @mac_01(x, 1, 2, 3, 4)

f_ma04 (generic function with 1 method)

In [35]:
f_ma04(1.5)
@benchmark f_ma04(1.5)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.001 ns … 0.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.100 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.067 ns ± 0.047 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  0.001 ns       Histogram: frequency by time        0.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

程序效率与手写的优化代码相当。

派生函数(generated functions)是与宏类似的函数定义方式， 可以起到类似手写优化代码的效果， 可以将原来一般性的循环写成直接的表达式。

## 关键字参数问题

Julia的多重派发是仅根据位置参数， 包括没有缺省值的和有缺省值的位置参数的类型来进行的； 关键字参数不影响多重派发， 这样， 用到关键字参数的函数就无法得到类型信息的好处， 效率相对而言比较低。 关键字参数适用于有很多选项的用户直接调用的参数， 如果是反复循环的对性能要求高的函数则应避免使用关键字参数。 例如：

In [36]:
f_kw01(x; y=2, z=2) = x^y + x^z
f_kw01(1.5, y=2, z=3)
@benchmark f_kw01(1.5, y=2, z=3)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  2.100 ns … 48.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.800 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.846 ns ±  1.122 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▅    █  ▆       ▅  ▇                                      
  ▅▁█▁▁▇▁█▁▁█▁▁▃▁▂▁▁█▁▁█▁▄▁▁▄▁▅▁▁▅▁▁▅▁▅▁▁▅▁▁▅▁▄▁▁▃▁▁▂▁▂▁▁▂▁▂ ▃
  2.1 ns         Histogram: frequency by time         4.3 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [37]:
f_kw01b(x, y=2, z=2) = x^y + x^z
f_kw01b(1.5, 2, 3)
@benchmark f_kw01b(1.5, 2, 3)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.001 ns … 0.600 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     0.100 ns             ┊ GC (median):    0.00%
 Time  (mean ± σ):   0.071 ns ± 0.047 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                                          █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▂
  0.001 ns       Histogram: frequency by time        0.1 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

两个版本性能相近， 可能是Julia做了优化。

# 基本计算

## 整数的表示与溢出

Julia的数值直接使用硬件的数值表示。 **整数值的计算不进行溢出检查， 所以利用整数值计算时需要用户自己注意溢出问题**，如：

In [38]:
2^62

4611686018427387904

In [39]:
2^63

-9223372036854775808

不进行溢出检查是为了程序效率必须付出的代价。

如果确实有可能发生溢出， Int128可以保存更多位， 进一步可以改用浮点数Float64， 或者用无限位数的整数类型BigInt， BigInt效率会很差。 如

In [40]:
BigInt(2)^63

9223372036854775808

即使用BigInt()说明， 也仅是将输入转换成了无限精度整型， 转换之前的溢出是没有控制的，比如：

In [41]:
BigInt(2^63)

-9223372036854775808

要注意， 直接写的整数根据数字位数， 较短时为Int类型， 超过Int范围首先选用Int64和Int128， 超过Int128后自动认为是BigInt类型。 如：

In [42]:
typeof(123456789012345678)

Int64

In [43]:
typeof(12345678901234567890)

Int128

In [44]:
typeof(1234567890123456789012345678901234567890)

BigInt

用parse(BigInt, "...")从字符串输入长整数，如

In [45]:
parse(BigInt, "12345678901234567890")

12345678901234567890

## 浮点数

现代的浮点数一般使用Float64格式， 并在内存中遵照IEEE754标准表示。 这样， 用Float64表示的浮点数可以高效地运算。

为了获得更高精度的浮点数， 可以用BigFloat类型， 这不像BigInt那样可以有任意精度， 而是有较高精度， 精度与舍入方式可以用函数setprecision()和setrounding()全局性地设置。

# 数组处理

科学计算， 包括统计建模、数据处理， 广泛使用数组、向量、矩阵运算， 计算中大量的时间都花费在对数组的处理上面。 需要构造高效的数组处理算法。

## 数组存储

数组通过指定元素类型为抽象类型可以保存不同类型的数组元素， 但是这样的数组就无法利用类型信息进行优化， 所以数组应尽可能保存同一种实在类型(concrete type)的元素。

数组类型为Array{T, N}， 其中T为元素类型， 一般是实在类型， N是维数。 比如Array{Float64, 2}表示浮点数矩阵。 这种数组可以高效地存储与处理。 实在类型的数组一般可以在连续的内存中存储， 使得其遍历或者随机访问都很便捷。 而其它的动态语言如Python一般需要使用指针指向每个数组元素， 除特定的定制数组类型以外。 连续存储的数组比用指针访问的数组效率要高得多。

Julia的多维数组当元素是基础类型时按照列优先原则保存， 比如矩阵的元素也在连续的内存中保存， 先保存第一列，再保存第二列，以此类推。 多维数组类似。

在遍历多维数组时按照其存储次序遍历效率更高； 遍历一个矩阵时， 按照列次序遍历效率更高，如：

In [46]:
A = [1 2 3; 4 5 6]
for j=1:size(A,2)
    for i=1:size(A,1)
        println("(", i, ", ", j, ") -> ", A[i,j])
    end
end

(1, 1) -> 1
(2, 1) -> 4
(1, 2) -> 2
(2, 2) -> 5
(1, 3) -> 3
(2, 3) -> 6


或

In [47]:
for j=1:size(A,2), i=1:size(A,1)
    println("(", i, ", ", j, ") -> ", A[i,j])
end

(1, 1) -> 1
(2, 1) -> 4
(1, 2) -> 2
(2, 2) -> 5
(1, 3) -> 3
(2, 3) -> 6


或

In [48]:
for i in eachindex(A)
    println(A[i])
end

1
4
2
5
3
6


下面的例子演示了按列次序与按行次序遍历矩阵的效率差别。 按列遍历的例子：

In [49]:
function f_ar01(A)
    s = zero(eltype(A))
    for j=1:size(A,2), i=1:size(A,1)
        s += A[i,j]^2
        A[i,j] = s
    end
    return s
end
f_ar01(rand(2,2))
A = rand(1_000,1_000)
@benchmark f_ar01(A)

BenchmarkTools.Trial: 2425 samples with 1 evaluation.
 Range (min … max):  1.294 ms …   5.578 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.948 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.051 ms ± 660.544 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂█▄          ▁            ▄                                  
  ███▇▆▄▄▅▅▇▇▇▇█▆▆▄▄▄▄▅▃▃▃▃██▆▄▃▂▂▂▂▃▂▃▂▂▂▂▂▂▂▁▂▂▁▁▂▂▁▁▁▁▂▁▁▃ ▃
  1.29 ms         Histogram: frequency by time        4.57 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

按行遍历的例子:

In [50]:
function f_ar01b(A)
    s = zero(eltype(A))
    for i=1:size(A,1), j=1:size(A,2)
        s += A[i,j]^2
        A[i,j] = s
    end
    return s
end
f_ar01b(rand(2,2))
@benchmark f_ar01b(A)

BenchmarkTools.Trial: 1666 samples with 1 evaluation.
 Range (min … max):  2.171 ms …   6.270 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.775 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.988 ms ± 706.787 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ▄▄█                                                       
  ▆▇████▆▅▄▄▃▃▄▃▄▅▄▄▄▄▃▄▄▅▄▄▄▄▃▃▄▅█▄▄▄▃▃▂▃▃▂▃▂▃▂▂▃▂▂▂▂▂▂▂▂▁▂▂ ▃
  2.17 ms         Histogram: frequency by time         5.1 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

在上面的例子中，按列遍历比按行遍历有明显的性能提升。

## 减少动态内存分配

对于在循环内反复执行的代码， 应避免重复地分配内存， 而是预先分配内存。 对于以数组为输入和输出的自定义函数， 可以将函数的输出也作为函数自变量。

如：

In [51]:
function f_ar02(x, y, n)
    local z
    for i in 1:n
        z = x + y
    end
    return z
end
f_ar02(rand(10), rand(10), 100)
@benchmark f_ar02(rand(10), rand(10), 10_000)

BenchmarkTools.Trial: 8332 samples with 1 evaluation.
 Range (min … max):  477.400 μs …   2.979 ms  ┊ GC (min … max): 0.00% … 28.75%
 Time  (median):     487.000 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   596.945 μs ± 228.360 μs  ┊ GC (mean ± σ):  2.32% ±  7.54%

  █▄▃▂▁  ▁▃  ▁ ▃▂     ▂        ▂▂           ▂▄▂                 ▁
  ██████████▆██████▆▅▆██▆▆▄▅█▇▇██▇▇▆▆▆▅▅▃▅▃▄█████▇▆▅▅▅▅▅▁▄▄▄▃▄▄ █
  477 μs        Histogram: log(frequency) by time       1.27 ms <

 Memory estimate: 1.37 MiB, allocs estimate: 10002.

进行了许多次内存分配。预先分配z的内存：

In [52]:
function f_ar02b(x, y, n)
    z = similar(x)
    for i in 1:n
        z .= x .+ y
    end
    return z
end
f_ar02b(rand(10), rand(10), 100)
@benchmark f_ar02b(rand(10), rand(10), 10_000)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  100.500 μs … 581.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     102.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   118.504 μs ±  38.757 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄▃▁▅▂▁ ▅▃▁ ▃▁▁                                ▄ ▂            ▂
  ███████████████▇▆█▇█▆▅▅▇▅▅▄▃▅█▇▄▄▄▃▃▄▃▁▄▁▃▁▁▃▁▃█▇█▇▆▅▅▃▅▄▄▅▅▅ █
  100 μs        Histogram: log(frequency) by time        254 μs <

 Memory estimate: 432 bytes, allocs estimate: 3.

后一个版本基本没有动态分配内存， 效率提升几倍。

## 用数组视图代替数组切片

设A为矩阵， A[:,j]表示A的第j列组成的一维数组（向量）， 这种子集访问称为数组切片（slice）。 对向量x的x[1:end-1]这样的子集也会制作副本（切片）。

**切片的问题是， A[:,j]会制作第j列的一个副本**， 如果这个副本被反复多次访问， 而且未复制之前的内容在内存中没有连续存储， 复制的做法可以利用复制后连续存储快速访问的优点使得程序效率较高。 但是， 如果副本仅一次性使用， 或者数据子集本来已经是连续存储的， 复制操作会造成严重的性能损失。

例如，如下的程序对矩阵求列和：

In [53]:
function f_ar03(A)
    nc = size(A,2)
    map(j -> sum(A[:,j]), 1:nc)
end
f_ar03(rand(10,10))
@benchmark f_ar03(rand(1000,1000))

BenchmarkTools.Trial: 817 samples with 1 evaluation.
 Range (min … max):  3.279 ms … 26.214 ms  ┊ GC (min … max):  0.00% … 46.40%
 Time  (median):     5.814 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   6.108 ms ±  2.408 ms  ┊ GC (mean ± σ):  18.45% ± 19.69%

   ▃█▃▄▃▇▁        ▁▃▁▁▁                                       
  ▇████████▆▅▆▆▇▇▇██████▆▇▄▅▅▄▄▅▅▄▅▃▃▄▄▁▃▃▃▃▃▄▃▃▄▃▃▃▂▁▁▁▂▁▁▃ ▄
  3.28 ms        Histogram: frequency by time        13.3 ms <

 Memory estimate: 15.39 MiB, allocs estimate: 1003.

上面的程序中用了许多次内存分配。

在访问矩阵的一列时最好不制作副本而直接访问矩阵存储的该列。 在用到矩阵切片的代码前面冠以@views宏就可以达到此目的。如

In [54]:
function f_ar03b(A)
    nc = size(A,2)
    @views map(j -> sum(A[:,j]), 1:nc)
end
f_ar03b(rand(10,10))
@benchmark f_ar03b(rand(1000,1000))

BenchmarkTools.Trial: 1325 samples with 1 evaluation.
 Range (min … max):  2.283 ms … 12.405 ms  ┊ GC (min … max):  0.00% … 60.70%
 Time  (median):     2.857 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.766 ms ±  1.906 ms  ┊ GC (mean ± σ):  18.45% ± 21.64%

  ██▄▃▃▄▃ ▁▃▂▁▁ ▂ ▅▂    ▂▃▁                                   
  █████████████▆█▇███▇▄██████▇█▇▅▅▆▅▆▅▆▁▅▄▄▄▅▆▁▅▆▄▁▅▁▅▅▅▅▄▆▆ █
  2.28 ms      Histogram: log(frequency) by time     11.1 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

第二个版本的运行效率明显提升， 内存分配减半。

## SIMD并行优化

在执行类似z[i] = x[i] + y[i]这样的向量对应元素运算时， 现代CPU提供了并行执行的SIMD指令， Julia可以自动分析代码对某些代码执行SIMD优化。 可以用@simd宏要求进行SIMD优化。 这样优化的循环必须是循环各步独立的， 循环体没有分支操作，循环次数固定， 下标是连续变化的。 如：

In [55]:
function f_ar04!(z, x, y)
    n = length(x)
    for i=1:n
        z[i] = x[i] + y[i]
    end
    return nothing
end
z = zeros(10)
f_ar04!(z, rand(10), rand(10))
z = zeros(1_000_000)
@benchmark f_ar04!(z, rand(1_000_000), rand(1_000_000))

BenchmarkTools.Trial: 607 samples with 1 evaluation.
 Range (min … max):  5.147 ms … 21.958 ms  ┊ GC (min … max):  0.00% … 49.81%
 Time  (median):     7.799 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.231 ms ±  3.216 ms  ┊ GC (mean ± σ):  16.83% ± 18.00%

  █▃▁ ▃▁▁ ▃▁  ▁ ▁▅▃▁▂       ▄▁                           ▁    
  ███▇██████▄▇████████▇▆▇█▅▇███▆▅▅▇▄▄▆▁▄▄▅▆▅▅▆▄▄▄▁▁▁▁▁▇▅▁█▅▄ ▇
  5.15 ms      Histogram: log(frequency) by time       18 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 4.

SIMD优化版本：

In [56]:
function f_ar04b!(z, x, y)
    n = length(x)
    @inbounds @simd for i=1:n
        z[i] = x[i] + y[i]
    end
    return nothing
end
z = zeros(10)
f_ar04b!(z, rand(10), rand(10))
z = zeros(1_000_000)
@benchmark f_ar04b!(z, rand(1_000_000), rand(1_000_000))

BenchmarkTools.Trial: 570 samples with 1 evaluation.
 Range (min … max):  5.353 ms … 19.099 ms  ┊ GC (min … max):  0.00% … 39.63%
 Time  (median):     9.063 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.762 ms ±  3.181 ms  ┊ GC (mean ± σ):  17.62% ± 18.56%

  █▅▂▁  ▄    ▂       ▅▄▂ ▆▄                                ▂  
  ████████▄▇██▇█▄▇▄█████▇██▅▆▅██▇▅▄▇▄▁▄█▁▄▅▄▁▅▅█▄▅▅▄▄▅▅█▇▆██ ▇
  5.35 ms      Histogram: log(frequency) by time     16.6 ms <

 Memory estimate: 15.26 MiB, allocs estimate: 4.

性能没有变化，可能是编译器本身已经包含了足够的优化。

## 数组遍历

在编写以数组为输入的函数时， 实际输入的数组可能是稠密数组、稀疏数组、数组的视图等。 这样，用单个下标遍历有可能降低性能。 数组一般提供eachindex()函数可以用来高效地线性遍历数组元素。

# 并行计算

Julia语言中直接提供了对并行计算的支持。 因为并行计算与硬件密切相关， 所以这些功能在不同运行环境中有些能发挥作用而有些受限。 **目前在非用户干预的情况下Julia还是单线程计算**， 例外是输入输出是非同步的， 一些内建的计算库如OpenBlas能够利用多线程。 为了实现并行计算需要用户自己调用并行计算功能。
统计计算中有大量重复计算可以利用并行计算加速， 尤其是随机模拟问题，比较适合用并行计算。

有一些问题可以不依赖于语言支持而并行执行， 比如， 一个模型的随机模拟研究， 往往需要对模型设置许多组真实参数， 然后对每一组真实参数产生许多批次的样本进行模型估计。 这样， 可以运行多个非交互的Julia程序， 每个程序中使用不同的参数模拟， 最后汇总结果。 这样做的好处是不需要了解并行计算， 缺点是需要人为地开始多组程序， 当某些组任务完成时再人为地开始新的任务。 当每个任务耗时都很长， 比如若干个小时的时候， 这样的方法还是可行的。

**最简单的并行运算是单一计算机的多个CPU核心(kernels或threads)同时独立地对一个数组的不同部分计算， 进一步还可以将若干台计算机组成集群(cluster)并行运行。 在统计中随机模拟经常需要对多个独立样本并行地执行一些计算， 这种情况下利用Julia的并行能力是有意义的， 即使单个计算机的4个核心同时运行能够将模拟速度提高一倍也是有意义的**。

Julia的**Distributed包**是标准库的一部分， 提供了这样的功能， 实际上是在不同核心、不同CPU、不同计算机上执行单独的进程， 称这样的每个进程为一个工作进程或者节点。

Julia提供了coroutine功能， 也是一种并行机制， 离散系统模拟SimJulia就是利用这种机制。

Julia的Base.Threads包试验性地提供了多线程功能， 功能尚未成熟。